In [46]:
from github import Github
# from commonjs_package_tokenizer import tokenize_package
import npm_crawler as npm
import os
import commonjs_package_tokenizer as t
import packages_term_indexer as pti_mod
import pandas as pd

In [59]:
%%time

# spider npm's "most starred repositories" page for
# package name and a github url.
packages = npm.get_most_starred_packages(max_pages=10)

# providing a github login means we can exceed the public API request limit of 60 requests per hour.
# We need to look up approximately 400 packages.
gh_user = os.environ.get('GITHUB_API_USER')
gh_token = os.environ.get('GITHUB_API_TOKEN')
gh = Github(gh_user, gh_token)

# for each of those packages, perform a shallow git clone
for name, gh_url in packages:
    clone_url = 'git://github.com/{}.git'.format(gh_url)
    clone_dir = '.cache/packages/{}'.format(name)
    
    # use github API to determine if this repo is predominantly javascript.
    # we want to exclude coffeescript/typescript repos because their tokens
    # are significantly different.
    is_javascript = gh.is_repo_javascript(gh_url)
    
    if is_javascript and not isdir(clone_dir):
        # git clone the repo. assign output to an unused variable to supress noise
        _ = !git clone $clone_url $clone_dir --depth=1

CPU times: user 652 ms, sys: 55.7 ms, total: 707 ms
Wall time: 1.06 s


In [81]:
%%time
reload(t)
reload(pti_mod)

# build a document-term matrix for all the packages we downloaded
pti = pti_mod.PackagesTermIndexer()

i = 0
for dir_name in os.listdir('.cache/packages'):
    i += 1
    if i > 30: break
    pkg = pti.register_package(dir_name)
    for token in t.tokenize_package(dir_name):
        pkg.register_term(token)

CPU times: user 13 s, sys: 430 ms, total: 13.4 s
Wall time: 15.6 s


In [82]:
%%time

tdm = pti.tdm()

CPU times: user 1.38 s, sys: 40.2 ms, total: 1.42 s
Wall time: 1.48 s


In [83]:
pd.DataFrame(tdm, columns=pti.keys())

,var,agenda,=,module.exports,require(./lib/agenda.js);,/*,Code,forked,from,https://github.com/rschmukler/agenda,...,(lastStart,value.substring(lastStart);,lastCharacter,lastValue[lastValue.length,(!includeSeparator,separator.test(lastCharacter),"lastValue.substring(0,",lastValue.length,tokens.push(lastValue);,tokens;
0,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0,...,0,0,0.000000,0,0,0,0,0,0,0
1,1.645962,1.584963,1.883321,0.519211,1,0.603942,0.694307,1,0.576653,1,...,0,0,0.000000,0,0,0,0,0,0,0
2,4.410944,0.000000,3.598467,0.862391,0,2.067652,0.000000,0,2.191104,0,...,0,0,0.000000,0,0,0,0,0,0,0
3,3.034572,0.000000,2.549971,1.261158,0,0.603942,0.000000,0,0.940481,0,...,0,0,0.000000,0,0,0,0,0,0,0
4,2.273396,0.000000,2.147254,1.014250,0,0.000000,0.000000,0,0.940481,0,...,0,0,0.000000,0,0,0,0,0,0,0
5,2.379365,0.000000,2.094096,0.519211,0,0.381045,0.000000,0,1.021394,0,...,0,0,0.000000,0,0,0,0,0,0,0
6,1.258763,0.000000,1.088916,0.259605,0,0.000000,0.000000,0,0.000000,0,...,0,0,0.000000,0,0,0,0,0,0,0
7,3.666105,0.000000,3.215074,1.780369,0,1.488702,0.000000,0,2.237448,0,...,0,0,0.000000,0,0,0,0,0,0,0
8,1.208730,0.000000,1.311315,0.259605,0,0.000000,0.000000,0,0.000000,0,...,0,0,0.000000,0,0,0,0,0,0,0
9,1.021496,0.000000,1.117384,0.411465,0,0.000000,0.000000,0,0.363828,0,...,0,0,0.000000,0,0,0,0,0,0,0
